In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.AdvancedSolvers;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
ArgumentException caught while opening database #2 at 'H:\weber': Database Error: field state directory (H:\weber\timesteps) does not exist.
Databases loaded: 
Capacity: 4
Count: 3



In [ ]:
string thisdir = Directory.GetCurrentDirectory();
string path = String.Concat(thisdir,@"\Benchmark_4",@"\Benchmark_4_2021-9-15"+".json");
var SubTab = TableExtensions.LoadFromFile(path);

In [ ]:
SubTab.GetColumnNames().Where(s=>s.Contains("Core"))

index,value
0,NoOfCores


In [ ]:
SubTab.Rows.Count

12

In [ ]:
public static class DoWhatUWant{

    public static PlotRowSelector Solver_DG_Proj(int Degree){
           return delegate (int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt){
    
               Nmn = "";
               Fmt = new PlotFormat();
                         
               
              
               LinearSolverCode SolverCode = (LinearSolverCode)Convert.ToInt32(Row["LinearSolver.SolverCode"]);
               int DGdegree   = Convert.ToInt32(Row["DGdegree:u"]);
               string PrjName = Convert.ToString(Row["ProjectName"]);
    
               switch(SolverCode){
                   case LinearSolverCode.exp_Kcycle_schwarz:
                       Nmn+="Kcycle w. add.-Schwarz";
                       //Fmt.LineColor = LineColors.Blue;
                       Fmt.PointType = PointTypes.LowerTriangle;
                       break;
                   case LinearSolverCode.exp_gmres_levelpmg:
                       Nmn+="GMRES w. pTG";
                       //Fmt.LineColor = LineColors.Yellow;
                       Fmt.PointType = PointTypes.Box;
                       break;
                   case LinearSolverCode.classic_pardiso:
                       Nmn+="Pardiso";
                       //Fmt.LineColor = LineColors.Red;
                       Fmt.PointType = PointTypes.OpenCircle;
                       Fmt.DashType  = DashTypes.Dotted;
                       break;
                   default:
                       Nmn = null;
                       Fmt = null;
                    return;
               }
                
               
               if(DGdegree!=Degree){
                    Nmn = null;
                    Fmt = null;
                    return;
               }
               switch(DGdegree){
                   case 2:
                     Nmn+=" DG2";
                     //Fmt.PointType = PointTypes.OpenDiamond;
                     break;
                   case 3:
                     Nmn+=" DG3";
                     //Fmt.PointType = PointTypes.OpenDiamond;
                     break;
                   case 5:
                     Nmn+=" DG5";
                     //Fmt.PointType = PointTypes.OpenDiamond;
                     break;
    
                   default:
                     Nmn = null;
                     Fmt = null;
                     return;
               }
                 
               Fmt.PointSize = 0.5;
               Fmt.LineWidth = 3;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineColor = LineColors.Black;
               //Fmt.DashType  = DashTypes.Solid;
           };
        
    }
    
    public static PlotRowSelectorEx Solver_DG_Proj(int DG){
        int Solver = LinearSolverCode.exp_Kcycle_schwarz;
        return delegate (int iSweep, int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt,
                     out double xValue, out double yValue) { 
               // - - - - - - - - - - - - - - - - - - - - - - - - 
               // PlotRowSelector:
               // selects, which table row goes to which graph,
               // and the respective color  
               // - - - - - - - - - - - - - - - - - - - - - - - - 
               int k           = Convert.ToInt32(Row["DGdegree:u"]);
               int solver_name = Convert.ToInt32(Row["LinearSolver.SolverCode"]);
              
               if(k != DG || solver_name != Solver) {
                    // degree does not match -> not in this plot
                    Nmn    = null;
                    Fmt    = null;
                    xValue = 0;
                    yValue = 0;
                    return;
               }
               
               /*
               BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.AddSol
               BoSSS.Solution.AdvancedSolvers.Schwarz.Solve
               BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.MinimizeResidual
               BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Residual
               BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Solve
               overlap_scaling
               block_solve_level
               caching
               nocaching
               */




               double mtxAssi  = Convert.ToDouble(Row["XdgMatrixAssembly"]);
               double aggBinit = Convert.ToDouble(Row["Aggregation_basis_init_time"]);
               double slvInit  = Convert.ToDouble(Row["Solver_Init_time"]);
               double slvRun   = Convert.ToDouble(Row["Solver_Run_time"]);
               double cores    = Convert.ToDouble(Row["NoOfCores"]);

               
               if(aggBinit > 700)
               Console.Write("aggBinit/slvRun = " + aggBinit + "/" + slvRun);
               
               Fmt           = new PlotFormat();
               Fmt.PointSize = 0.5;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineColor = LineColors.Black;
               Fmt.LineWidth = 3;
               
               xValue = cores;
               
               switch(iSweep) {
                   case 0:
                   yValue       = slvRun;
                   Nmn          = "Slv Iter";
                   Fmt.DashType = DashTypes.Dotted;
                   Fmt.PointType = PointTypes.OpenCircle;
                   break;
                   
                   case 1:
                   yValue        = slvInit;
                   Nmn           = "Slv Init";
                   Fmt.DashType  = DashTypes.Dashed;
                   Fmt.PointType = PointTypes.OpenBox;
                   break;
                   
                   case 2:
                   yValue        = aggBinit;
                   Nmn           = "Agg Init";
                   Fmt.DashType  = DashTypes.DotDashed;
                   Fmt.PointType = PointTypes.OpenDiamond;
                   break;
                   
                   case 3:
                   yValue        = mtxAssi;
                   Nmn           = "Mtx ass";
                   Fmt.DashType  = DashTypes.DotDotDashed;
                   Fmt.PointType = PointTypes.OpenUpperTriangle;
                   break;
                   
                   default:
                   throw new NotImplementedException();
               }
               
               if(aggBinit > 700)
               Console.WriteLine("\t + case: {0} + yValue = {1}", iSweep, yValue);
           };
    }
}
    

Error: (153,6): error CS1513: } expected